# Labeling

In [1]:
import os
from tqdm import tqdm
import pandas as pd

In [65]:
def label():

    # Concat 데이터 불러오기
    files_list = os.listdir('../data/20221121/concat/')
    
    for file_name in tqdm(files_list[1732:1733]):

        stock_info = pd.read_csv(f'../data/20221121/concat/{file_name}', index_col=0, encoding='utf-8-sig')
        
        code = 'A' + file_name.split('_')[-2]

        # 데이터 프레임에서 날짜 인덱스 추출
        date = stock_info.index.unique()

        # label 컬럼 추가 후 0으로 초기화
        stock_info['label'] = 0

        # 업데이트에 사용할 데이터 프레임 생성
        update_stock_info = pd.DataFrame()

        print(file_name)

        # labeling
        for day in date:

            # 특정일의 Data 추출
            select_day = stock_info.loc[day].copy()
            select_day['label'] = 0
            
            # 특정일의 Row 만큼 반복
            for row in range(len(select_day)):
                
                # 특정일의 현재 row 이후 최대 고가를 추출
                next_price = select_day[-row-1::-1]['고가'].max()

                # 추출한 최대 고가를 label 컬럼에 대입
                select_day.iloc[-row-1,-1] = next_price
                next_price = 0
            
            # 특정일 label이 추가된 DF를 업데이트할 DF에 concat
            update_stock_info = pd.concat([update_stock_info, select_day])

        trans = update_stock_info.loc[:,['고가','label']]
        trans = trans.rename(columns={'label':'pct_label'}).T

        get_trans = trans.pct_change().T.iloc[:,-1]
        
        update_stock_info = pd.concat([update_stock_info, get_trans], axis=1)
        update_stock_info = update_stock_info.drop(['label'], axis=1)

        update_stock_info['pct_label'] = update_stock_info['pct_label'].mul(100)
        update_stock_info['pct_label'] = update_stock_info['pct_label'].round(1)

        print(file_name)

        update_stock_info.to_csv('../data/20221121/complete/{0}.csv'.format(code), encoding='utf-8-sig')

In [66]:
label()

  0%|          | 0/1 [00:00<?, ?it/s]

concat_189330_씨이랩.csv
concat_189330_씨이랩.csv


100%|██████████| 1/1 [00:38<00:00, 38.51s/it]
